In [1]:
!git clone https://github.com/srijandas07/video_convolution.git


Cloning into 'video_convolution'...
remote: Enumerating objects: 808, done.
remote: Total 808 (delta 0), reused 0 (delta 0), pack-reused 808
Receiving objects: 100% (808/808), 372.06 MiB | 36.81 MiB/s, done.
Resolving deltas: 100% (22/22), done.
Checking out files: 100% (754/754), done.


In [2]:
!ls


sample_data  video_convolution


In [3]:
cd video_convolution/scripts/

/content/video_convolution/scripts


In [4]:
!pwd

/content/video_convolution/scripts


In [5]:
import numpy as np, pandas as pd, os, sys
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Model


In [6]:
from keras.applications.vgg16 import VGG16
#To_Do: load VGG model
model = VGG16(weights='imagenet',include_top=True)

553476096/553467096 [==============================] - 4s 0us/step


In [7]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
#To_Do: create a model using VGG16 architecture to perform feature extraction
model = Model(inputs=model.input, outputs=model.get_layer('fc2').output)
f_size = 4096

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
file_path = '/content/video_convolution/data/p1_arrangingobjects_subAction510175554/'                           
n_files = sorted(os.listdir(file_path))
f_p = os.path.basename(os.path.normpath(file_path))
batch_size = 50
k = 0

In [11]:
def data_load_batch(k, batch_size):
    t_file = []
    for img_file in n_files[0+k:batch_size+k]:
        img_path = file_path+'/'+img_file
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        t_file.append(preprocess_input(x))
    t_file = np.asarray(t_file)
    return t_file



In [12]:
images = []
features_conv = []

for i in range(0, int(len(n_files)/batch_size)):
    images = data_load_batch(k, batch_size)
    #To_Do: extract features from the created model providing batch as input
    features_conv.append(model.predict(images,batch_size=batch_size))
    k = k + batch_size




In [13]:
features_conv = np.asarray(features_conv)
features_conv.shape


(4, 50, 4096)

In [14]:
features_conv = np.reshape(features_conv, [features_conv.shape[0]*features_conv.shape[1], f_size])
features_conv.shape

(200, 4096)

In [15]:
images_remain = len(n_files) - k
features_conv5_remain = []
if images_remain > 0:
   images = data_load_batch(k, images_remain)
   features_conv5_remain.append(model.predict(images, batch_size=images_remain))
features_conv5_remain = np.asarray(features_conv5_remain)                           
features_conv5_remain = np.squeeze(features_conv5_remain)
features_conv = np.vstack([features_conv, features_conv5_remain])
features_conv.shape

(232, 4096)

In [16]:
np.savetxt('./'+f_p+'.csv.gz', features_conv, delimiter=',')

In [17]:
def max_min_conv(video):
    frame_features  = np.loadtxt(video, delimiter=',')
    max_features = np.amax(frame_features, axis=0)
    min_features = np.amin(frame_features, axis=0)
    final_t1 = np.hstack([max_features, min_features])
    return final_t1

In [18]:
import glob
path = './'
for video in glob.glob(path+'*.csv.gz'):
    desc = []
    features = max_min_conv(os.path.join(path, video))
    desc = np.hstack([desc, features.ravel()])
    np.savetxt('/content/video_convolution/results/video_descriptors/'+os.path.basename(video)+'.txt', desc, delimiter=',')